# 1. Import Packages

In [1]:
import numpy as np
from scipy.integrate import odeint
import plotly.express as px
import plotly.graph_objects as go

# 2. Setup Differential Equations

In [2]:
def derive(y, t, N, alpha, gamma, delta, epsilon, mu, nu, rho, R_0, start, end, ppl, ppo, startVac, ppv):
    S, E, I, R, D, L = y
    S_0 = N-1
    dSdt = (epsilon * N) + (gamma * R) + (xi(t, end, ppo) * L) + (-beta(t, R_0, S, S_0, nu) * S * I / N) + (-mu * S) + (-theta(start, t, end, ppl) * S) + (-phi(t, startVac, ppv) * S)
    dEdt = (beta(t, R_0, S, S_0, nu) * S * I / N) + (-mu * E) + (-delta * E) 
    dIdt = (delta * E) + (-mu * I) + (-rho * alpha * I) + (-nu * (1 - alpha) * I)
    dRdt = (nu * (1 - alpha) * I) + (-mu * R) + (-gamma * R) + (phi(t, startVac, ppv) * S)
    dDdt = (alpha * rho * I)
    dLdt = (theta(start, t, end, ppl) * S) + (-xi(t,end,ppo) * L) + (-mu * L)
    return dSdt, dEdt, dIdt, dRdt, dDdt, dLdt

def beta(t, R_0, S, S_0, nu):
        return (R_0 * S / S_0) * nu
    
def theta(start, t, end, ppl):
    if (t >= start) & (t <= end):
        return ppl
    else:
        return 0
# if ppl = 0 => No lockdown

def xi(t, end, ppo):
    if (t > end):
        return ppo
    else:
        return 0
# After lockdown, again ppo = 0 => No lockdown

def phi(t, startVac, ppv):
    if (t > startVac):
        return ppv
    else:
        return 0
    

# 3. Calculation Function

In [3]:
def UInput(Pop, R0, InfPeriod, IncPeriod, InfDeath, DaysToDeath, ImmLoss, QuStartDay, QuEndDay, ppl, ppo, startVac, ppv):
    N = Pop
    
    epsilon = 0.0118 / 365 # Birth rate in the U.S. = 11.8/1000
    mu = 0.00864 / 365 # Natural Death Rate = 863.8 deaths per 100,000 population
    
    psi = InfPeriod # Infectious period
    nu = 1.0 / psi
    
    delta = 1.0 / IncPeriod  # incubation period of five days
    
    R_0 = R0
    start = QuStartDay # lockdown start date
    end = QuEndDay
    
    gamma = ImmLoss # Recovered->Susceptible 

    alpha = InfDeath # Rate of death due to infection
    rho = 1 / DaysToDeath # 9 days from infection until death

    S0, E0, I0, R0, D0, L0 = N-1, 1, 0, 0, 0, 0  # initial conditions: one exposed
    
    t = np.linspace(0, 365, 365) # Grid of time points (in days)
    y0 = S0, E0, I0, R0, D0, L0 # Initial conditions

    # Integrate SEIRDL equations over time t.
    result = odeint(derive, y0, t, args=(N, alpha, gamma, delta, epsilon, mu, nu, rho, R_0, start, end, ppl, ppo, startVac, ppv))
    S, E, I, R, D, L = result.T
    
    
    return t, E, I, D


# 4. Plot functions / User Input:

In [4]:
def plotseirdl1(Pop, InfPeriod, IncPeriod, InfDeath, DaysToDeath, ImmLoss, QuStartDay, QuEndDay, ppl, ppo, startVac, ppv):
    fig = go.Figure(layout = go.Layout(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'))
    
    for R0 in np.arange(0, 6, 0.1):
        # Create and style traces
        t, E, I, D = UInput(Pop, R0, InfPeriod, IncPeriod, InfDeath, DaysToDeath, ImmLoss, QuStartDay, QuEndDay, ppl, ppo, startVac, ppv)
        
        fig.add_trace(go.Scatter(visible=False, x=t, y=E, name = 'Exposed', line=dict(color='gold', width=2)))
        fig.add_trace(go.Scatter(visible=False, x=t, y=I, name='Infected', line=dict(color='red', width=2)))
        fig.add_trace(go.Scatter(visible=False, x=t, y=D, name='Dead', line = dict(color='black', width=2)))
    
   
    fig.data[0].visible = True
    
    # Create and add slider
    steps = []
    for i in range(0, len(fig.data), 3):
        ra = np.arange(0, 6, 0.0335)
        step = dict(label = round(ra[i],2),method="update", args=[{"visible": [False] * len(fig.data)}])
        step["args"][0]["visible"][i] = True
        step["args"][0]["visible"][i+1] = True 
        step["args"][0]["visible"][i+2] = True# Toggle i'th trace to "visible"
        steps.append(step)
    
    
    sliders = [dict(active=90, currentvalue={"prefix": "R0 Level: "}, steps=steps)]

    fig.update_layout(sliders=sliders)

    # Edit the layout
    fig.update_layout(xaxis_title='time (Days)')

    fig.show()
    
    # Uncomment below when you need to save the plot
    #fig.write_html("/Users/maharshi/Desktop/thalamos/Covid/Web/HTML/R0Slider.html")
    


In [5]:
from ipywidgets import interact

In [6]:
xs=np.linspace(0, 6, 100)

fig = go.FigureWidget([go.Scatter(), go.Scatter(),go.Scatter()])

fig.update_layout(xaxis_title='Time (Days)')
fig.update_layout(paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')

#UInput(Pop, R0, InfPeriod, IncPeriod, InfDeath, DaysToDeath, ImmLoss, QuStartDay, QuEndDay, ppl, ppo, startVac, ppv):
@interact(R0=(1.0, 6.0, 0.1), InfPeriod=(1.0,30.0,1.0), IncPeriod=(1.0,30.0,1.0), InfDeath=(0.001,0.5,0.001),
          DaysToDeath=(1.0,30.0,1.0), ImmLoss = (0.001,0.5,0.001), QuStartDay = (0.0,365.0,1.0),QuEndDay=(0.0,365.0,1.0),
          ppl=(0.01,1.0,0.01), ppo=(0.01,1.0,0.01), startVac=(0.0,365.0,1.0), ppv=(0.01,1.0,0.01))
def update(R0=3.0, InfPeriod=4.0, IncPeriod=8.0, InfDeath=0.055, DaysToDeath=9.0, ImmLoss=0.02,
           QuStartDay=50.0, QuEndDay=150.0, ppl=0.5, ppo=0.9, startVac=200.0, ppv=0.7):
    with fig.batch_update():
        t, E, I, D = UInput(170_000, R0, InfPeriod, IncPeriod, InfDeath, DaysToDeath, ImmLoss, QuStartDay, QuEndDay, ppl, ppo, startVac, ppv)
        fig.data[0].x=t
        
        fig.data[0].y=E
        fig.data[0].line.color='gold'
        fig.data[0].name = "Exposed"
        
        fig.data[1].y=I
        fig.data[1].line.color='red'
        fig.data[1].name = "Infected"
        
        fig.data[2].y=D
        fig.data[2].line.color='black'
        fig.data[2].name = "Dead"

fig

interactive(children=(FloatSlider(value=3.0, description='R0', max=6.0, min=1.0), FloatSlider(value=4.0, descr…

FigureWidget({
    'data': [{'line': {'color': 'gold'},
              'name': 'Exposed',
              'type':…

# 5. Test

In [11]:
import nbinteract as nbi
nbi.publish('maharshipandya/ThalamosAI/home', 'CovidSim-010.ipynb')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Saving notebook... Saved 'CovidSim-010.ipynb'.
Converting notebook...



Successfully converted!

<a href="CovidSim-010.html" target="_blank" download>Click to download your webpage.</a>

To host your webpage, see the documentation:
<a href="https://www.nbinteract.com/tutorial/tutorial_publishing.html"
        target="_blank">
    https://www.nbinteract.com/tutorial/tutorial_publishing.html
</a>
